In [61]:
import pandas as pd
import os
from pymongo import MongoClient

In [62]:


def file_route(directory):
    
    df_list = []
    
   
    try:
        files = os.listdir(directory)
    except Exception as e:
        print(f"디렉토리 접근 중 오류 발생: {e}")
        return None
    
    
    for file in files:
        if '주식거래 수수료' in file:
            file_path = os.path.join(directory, file)
            try:
                
                df = pd.read_excel(file_path, header=2)  
                df_list.append(df)
                print(f"{file_path} 파일 읽기 성공")
            except Exception as e:
                print(f"{file_path} 파일을 읽는 중 오류 발생: {e}")
    
    # 데이터프레임 병합
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        return merged_df
    else:
        print("읽은 파일이 없습니다.")
        return None


# 디렉토리 경로 (예시)
directory = '../datasets/'

# 함수 호출하여 병합된 데이터프레임 반환
merged_data = file_route(directory)

# 병합된 데이터 출력 (있으면)
if merged_data is not None:
    print(merged_data.head())



../datasets/주식거래 수수료_10.xls 파일 읽기 성공
../datasets/주식거래 수수료_1.xls 파일 읽기 성공
../datasets/주식거래 수수료_20241217 .xls 파일 읽기 성공
../datasets/주식거래 수수료_1000.xls 파일 읽기 성공
../datasets/주식거래 수수료_50.xls 파일 읽기 성공
../datasets/주식거래 수수료_100.xls 파일 읽기 성공
../datasets/주식거래 수수료_500.xls 파일 읽기 성공
      회사명             수수료\n부과기준        기준일자  거래금액   구분   오프라인     HTS    ARS  \
0  유진투자증권  /img/sub/ico_xls.gif  2012/10/08  10만원  변경전    NaN     NaN    NaN   
1  유진투자증권  /img/sub/ico_xls.gif  2020/06/17  10만원  변경후  500.0   180.0  200.0   
2  유진투자증권  /img/sub/ico_xls.gif  2020/06/17  10만원  변경율    NaN     NaN    NaN   
3    SK증권  /img/sub/ico_xls.gif  2011/11/18  10만원  변경전  500.0  1150.0  250.0   
4    SK증권  /img/sub/ico_xls.gif  2015/06/01  10만원  변경후  500.0  1150.0  250.0   

     스마트폰  오프라인.1  HTS.1  ARS.1  스마트폰.1              비고  
0     NaN     NaN    NaN    NaN    15.0             NaN  
1   150.0   500.0   15.0  200.0    15.0             NaN  
2     NaN     NaN    NaN    NaN     0.0             NaN  
3    50.0   500.0 

In [63]:

# # 데이터 병합
# merged_df = pd.concat(merged_data, ignore_index=True)

In [64]:

# 병합된 데이터 CSV로 저장
merged_data.to_csv('merged_data2.csv', index=False)


In [65]:


client = MongoClient('python_jupyternote_mongo-mongodb-1')  
db = client['pandas_db']  
collection = db['stocktradingfees_merge']  


data_dict = merged_data.to_dict(orient='records')  
collection.insert_many(data_dict)  


InsertManyResult([ObjectId('6761307fb128bcafdae29bbc'), ObjectId('6761307fb128bcafdae29bbd'), ObjectId('6761307fb128bcafdae29bbe'), ObjectId('6761307fb128bcafdae29bbf'), ObjectId('6761307fb128bcafdae29bc0'), ObjectId('6761307fb128bcafdae29bc1'), ObjectId('6761307fb128bcafdae29bc2'), ObjectId('6761307fb128bcafdae29bc3'), ObjectId('6761307fb128bcafdae29bc4'), ObjectId('6761307fb128bcafdae29bc5'), ObjectId('6761307fb128bcafdae29bc6'), ObjectId('6761307fb128bcafdae29bc7'), ObjectId('6761307fb128bcafdae29bc8'), ObjectId('6761307fb128bcafdae29bc9'), ObjectId('6761307fb128bcafdae29bca'), ObjectId('6761307fb128bcafdae29bcb'), ObjectId('6761307fb128bcafdae29bcc'), ObjectId('6761307fb128bcafdae29bcd'), ObjectId('6761307fb128bcafdae29bce'), ObjectId('6761307fb128bcafdae29bcf'), ObjectId('6761307fb128bcafdae29bd0'), ObjectId('6761307fb128bcafdae29bd1'), ObjectId('6761307fb128bcafdae29bd2'), ObjectId('6761307fb128bcafdae29bd3'), ObjectId('6761307fb128bcafdae29bd4'), ObjectId('6761307fb128bcafdae29b

In [66]:
query = {'구분': '변경후'}
cursor = collection.find(query)

In [67]:
# 데이터를 DataFrame으로 변환
data = pd.DataFrame(list(cursor))

In [68]:
# '스마트폰.1' 컬럼에 대한 평균 계산
smartphone_avg = data['스마트폰.1'].mean()
smartphone_avg

16393.557575757575

In [69]:
# 스마트폰 수수료 평균 이상인 증권사들만 필터링
df_filtered = data[data['스마트폰.1'] >= smartphone_avg]
df_filtered

,_id,회사명,수수료\n부과기준,기준일자,거래금액,구분,오프라인,HTS,ARS,스마트폰,오프라인.1,HTS.1,ARS.1,스마트폰.1,비고
1,67611df0b140033f893c1fb9,SK증권,/img/sub/ico_xls.gif,2015/06/01,1억원,변경후,500000.0,120000.0,250000.0,120000.0,500000.0,15000.0,250000.0,120000.0,증권통App 수수료율 변경
2,67611df0b140033f893c1fbc,유안타증권,/img/sub/ico_pdf.gif,2021/01/04,1억원,변경후,500000.0,65000.0,197000.0,100000.0,500000.0,14000.0,197000.0,100000.0,NaN
6,67611df0b140033f893c1fc7,상상인증권,/img/sub/ico_pdf.gif,2020/04/23,1억원,변경후,490000.0,140000.0,200000.0,140000.0,490000.0,140000.0,200000.0,140000.0,NaN
7,67611df0b140033f893c1fc9,대신증권,/img/sub/ico_pdf.gif,2021/01/01,1억원,변경후,472296.0,97296.0,197296.0,197296.0,472296.0,97296.0,197296.0,197296.0,NaN
8,67611df0b140033f893c1fcc,교보증권,/img/sub/ico_pdf.gif,2021/09/01,1억원,변경후,499200.0,99200.0,249200.0,149200.0,499200.0,99200.0,249200.0,149200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,6761307fb128bcafdae29c53,부국증권,/img/sub/ico_hwp.gif,2011/11/18,1억원,변경후,445377.0,95377.0,195377.0,150000.0,445377.0,95377.0,195377.0,150000.0,NaN
1216,6761307fb128bcafdae29c6b,IBK투자증권,/img/sub/ico_hwp.gif,2013/02/04,1억원,변경후,500000.0,100000.0,250000.0,100000.0,500000.0,100000.0,250000.0,100000.0,NaN
1223,6761307fb128bcafdae29c7e,비엔케이투자증권,/img/sub/ico_pdf.gif,2020/08/24,1억원,변경후,449164.0,79164.0,NaN,150000.0,449164.0,14164.0,NaN,150000.0,NaN
1267,6761307fb128bcafdae29cf4,대신증권,/img/sub/ico_pdf.gif,2021/01/01,1000만원,변경후,49730.0,15730.0,19730.0,19730.0,49730.0,15730.0,19730.0,19730.0,NaN


In [70]:
#증권사별 스마트폰 수수료 평균 
df_grouped = df_filtered.groupby('회사명')['스마트폰.1'].mean().reset_index()
df_grouped

,회사명,스마트폰.1
0,DB금융투자,99164.0
1,IBK투자증권,100000.0
2,KB증권,120000.0
3,NH투자증권,108440.0
4,SK증권,120000.0
5,교보증권,149200.0
6,대신증권,108513.0
7,메리츠증권,88132.0
8,부국증권,150000.0
9,비엔케이투자증권,150000.0


In [71]:
# 전체 스마트폰 수수료 평균
overall_avg = smartphone_avg
overall_avg

16393.557575757575

In [72]:

df_grouped.to_csv('smartphone_fee_avgup2.csv', index=False)